# ----------------------------------------------------------------------------------
### Reader note: This Notebook is an add-on to 'Wine Preprocessing and Linear Regression' Notebook.
### Export of 'residuals x quality df' is at bottom of this Notebook
### ----------------------------------------------------------------------------------

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import cleaning_functions as cf 

from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse, mean_absolute_error as mae, r2_score
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import cross_val_score
import statsmodels.api as sm

In [3]:
combined_file_path = '../data/cleaned/winequality_combined_cleaned.csv' 
red_file_path = '../data/cleaned/winequality_red_cleaned.csv'
white_file_path = '../data/cleaned/winequality_white_cleaned.csv'
combined_df = pd.read_csv(combined_file_path)
red_df = pd.read_csv(red_file_path, index_col=0)
white_df = pd.read_csv(white_file_path, index_col=0)
red_df

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.660,0.00,1.8,0.075,13.0,40.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1354,6.8,0.620,0.08,1.9,0.068,28.0,38.0,0.99651,3.42,0.82,9.5,6
1355,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1356,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1357,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [7]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.25)
    quartile3 = dataframe[variable].quantile(0.75)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = round(low_limit,5)
    dataframe.loc[(dataframe[variable] > up_limit), variable] = round(up_limit,5)


In [8]:
red_df.columns

Index(['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar',
       'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [9]:
columns_gpt = ['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar',
       'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'sulphates']# only columns identified as having outliers

columns_matthew = ['citric_acid', 'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'pH', 'sulphates']

columns_combined = ['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar',
       'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'pH', 'sulphates']

columns_all = ['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar',
       'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density',
       'pH', 'sulphates', 'alcohol']

In [10]:
for col in columns_matthew:
    replace_with_thresholds(red_df, col)

In [11]:
red_df

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.660,0.00,1.8,0.075,13.0,40.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1354,6.8,0.620,0.08,1.9,0.068,28.0,38.0,0.99651,3.42,0.82,9.5,6
1355,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1356,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1357,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [12]:
#define X and Y
X = red_df.drop(['quality', 'free_sulfur_dioxide'], axis=1)
#X = pd.DataFrame(red_df[['citric_acid', 'total_sulfur_dioxide', 'density', 'pH', 'residual_sugar', 'chlorides', 'sulphates', 'alcohol']])
y = pd.DataFrame(red_df["quality"])

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42)

In [13]:
scaler_s_x = StandardScaler()
scaler_s_x.fit(X_train)
X_train_s = scaler_s_x.transform(X_train)
X_test_s = scaler_s_x.transform(X_test)

In [14]:
# Linear regression
lm = LinearRegression()
model = lm.fit(X_train_s, y_train)
print(f'model coefficients:\n {model.coef_}\n')
print(f'model intercept:\n {model.intercept_}\n')

# Applying model to X test
y_pred = model.predict(X_test_s)
#y_pred = pd.DataFrame(scaler_s_y.inverse_transform(y_pred)) # inversing y
y_pred = pd.DataFrame(y_pred)

y_pred = y_pred.rename(columns = {0:"y_pred"})

y_test = y_test.reset_index(drop=True)
y_test = y_test.rename(columns = {"quality":"y_test"})

residuals_df = pd.concat([y_test,y_pred], axis = 1)
residuals_df["residual"] = residuals_df["y_test"] - residuals_df["y_pred"]


# Root mean squared error
rmse = mse(y_test, residuals_df["y_pred"], squared=False)
print(f'Root mean squared error: {rmse} \n')

# R^2
r2 = r2_score(y_test, residuals_df["y_pred"])
print(f'R2: {r2} \n')

# Calculating adjusted R^2
n = X_train.shape[0]  # Number of observations in the training set
p = X_train.shape[1]  # Number of features used for training
adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
print(f'Adjusted R2: {adjusted_r2} \n')

residuals_df

model coefficients:
 [[-0.03909706 -0.17360898 -0.03821616  0.00447879 -0.09930921 -0.08418242
   0.04110446 -0.11910698  0.17115604  0.31595454]]

model intercept:
 [5.63199215]

Root mean squared error: 0.6317195350864011 

R2: 0.4095500837218208 

Adjusted R2: 0.4036924456635055 



,y_test,y_pred,residual
0,5,5.172826,-0.172826
1,6,5.839787,0.160213
2,7,6.435709,0.564291
3,5,5.109624,-0.109624
4,4,5.118868,-1.118868
...,...,...,...
335,5,5.697738,-0.697738
336,6,5.904275,0.095725
337,6,5.681775,0.318225
338,7,6.539583,0.460417


In [15]:
quality_column = red_df['quality']

In [17]:
df_residuals_quality = pd.concat([residuals_df, quality_column], axis=1)

In [31]:
df_residuals_quality = df_residuals_quality.dropna()

In [32]:
df_residuals_quality

,y_test,y_pred,residual,quality
0,5.0,5.172826,-0.172826,5
1,6.0,5.839787,0.160213,5
2,7.0,6.435709,0.564291,5
3,5.0,5.109624,-0.109624,6
4,4.0,5.118868,-1.118868,5
...,...,...,...,...
335,5.0,5.697738,-0.697738,5
336,6.0,5.904275,0.095725,6
337,6.0,5.681775,0.318225,6
338,7.0,6.539583,0.460417,6


In [33]:
df_residuals_quality.to_csv('../data/analysis/df_residuals_quality.csv',index=False)